In [6]:
import json
import copy
import math
import numpy as np
from scipy.stats import norm

def CalculateScore(statistical_variable,value):
	score_values = {"low":0.3,
						"high":0.7}
	level = LevelOfStatisticalVariable(statistical_variable,(-1)*value)
	if level.keys() == ["low"] :
		score = score_values["low"] * ((value-level["low"][0])
						/(level["low"][1]-level["low"][0]))
	elif level.keys() == ["moderate"] :
		score = score_values["high"]-score_values["low"] * ((value-level["moderate"][0])
						/(level["moderate"][1]-level["moderate"][0]))+score_values["low"]
	else:
		score = 1-score_values["high"] * ((value-level["high"][0])
						/(level["high"][1]-level["high"][0]))+score_values["high"]
	return score



def CalculateTotalRisk(volatility_val,Var_val,Cvar_val):
	volatility_score = CalculateScore("standard_deviation",volatility_val)
	Var_score = CalculateScore("Var",Var_val)
	Cvar_score = CalculateScore("Cvar",Cvar_val)
	total_risk = (0.5 * volatility_score) + (0.4 * Var_score) + (0.1 * Cvar_score)
	return total_risk




def LevelOfStatisticalVariable(statistical_variable,value):
	standard_deviation = [{"low" : (0,0.05)},
												{"moderate" : (0.05,0.15)},
												{"high" : (0.15,1)}]
	Var = [{"low" : (0,0.02)},
				 {"moderate" : (0.02,0.05)},
				 {"high" : (0.05,1)}]
	standard_deviation = [{"low" : (0,0.03)},
												{"moderate" : (0.03,0.06)},
												{"high" : (0.06,1)}]
	if statistical_variable == "standard_deviation":
		if value <= 0.05:
			return standard_deviation[0]
		if value > 0.05 and value < 0.15 :
			return standard_deviation[1]
		else:
			return standard_deviation[2]

	if statistical_variable == "Var":
		value = value * (-1)
		if value <= 0.02:
			return standard_deviation[0]
		if value > 0.02 and value < 0.05 :
			return standard_deviation[1]
		else:
			return standard_deviation[2]

	if statistical_variable == "Cvar":
		value = value * (-1)
		if value <= 0.03:
			return standard_deviation[0]
		if value > 0.03 and value < 0.06 :
			return standard_deviation[1]
		else:
			return standard_deviation[2]




def get_var_and_cvar(stock, confidence_level=0.95):
    """
    Calculate Value at Risk (VaR) and Conditional Value at Risk (CVaR).

    Args:
        stock (dict): Dictionary containing 'symbol' and 'prices'.
        confidence_level (float): Confidence level for VaR (e.g., 0.95 for 95%).

    Returns:
        tuple: VaR and CVaR values.
    """
    prices= stock["prices"]
    returns= calculate_returns(prices)

    z_score= norm.ppf(1- confidence_level)# Z-score for the given confidence levelmean= np.mean(returns)
    std_dev= np.std(returns)
    var= mean- z_score* std_dev# VaR formula# Calculate CVaR as the mean of losses exceeding VaRlosses_beyond_var= returns[returns<= var]
    cvar= np.mean(losses_beyond_var)if len(losses_beyond_var)> 0 else var

    return var, cvar# Return raw VaR/CVaR (negative for losses)




def GetStandarddeviation(stock):
	import json
	import numpy as np
	with open('\Data\stocks_consecutive_day_volatility.json','r') as file :
		data = json.load(file)
		symbol = stock["symbol"]
		stock_dict = next((item for item in data if item['symbol'] == symbol),None)  #next() finds the first match
		volatilities = stock_dict['volatilities']
		std_value = np.std(volatilities,ddof=1) #ddof is the data degree of freedom
		return std_value





def GetPortfolioStatVariables(portfolio,action): #update other functions to take a stock not a node
    stocks = portfolio.stocks
    quadratic_sum = 0
    var_sum = 0
    cvar_sum = 0
    stat_variables={}
    for stock in stocks :
      std_dev = GetStandarddeviation(stock)
      var = get_var_and_cvar(stock)[0]
      cvar = get_var_and_cvar(stock)[1]
      weight = GetWeight(stock,action,portfolio)
      quadratic_sum += (weight ** 2) * (std_dev ** 2)
      var_sum += weight *  var
      cvar_sum += weight * cvar
    stat_variables["standard_deviation"] = math.sqrt(quadratic_sum)
    stat_variables["Var"] = var_sum
    stat_variables["Cvar"] = cvar_sum
    return stat_variables




def GetWeight(stock ,portfolio,action="None"):
	if action == "buy" :
		weight = (stock["price"]*(stock["shares"]+1)) / portfolio.funds
	elif action == "sell":
		weight = (stock["price"]*(stock["shares"]-1)) / portfolio.funds
	else :
		weight = (stock["price"]*(stock["shares"])) / portfolio.funds
	return weight





def stock_class_limit_violated(Risky ,moderate ,low , portfolio_type, mainstock,allocation, stock_class):
    risk_allocation_limit = {
     "risky" : {
         "high":   (0.50 , 0.70),
         "moderate":   (0.20 , 0.40),
         "low":   (0.00 , 0.15),
      },
      "moderate" : {
         "high":   (0.20 , 0.35),
         "moderate":   (0.4 , 0.50),
         "low":   (0.15 , 0.30),
      },
      "lowrisk" : {
         "high":   (0.00 , 0.10),
         "moderate":   (0.20 , 0.40),
         "low":   (0.50 , 0.70),
      }
      }
    if portfolio_type == "risky":
       if mainstock in Risky :#imported from data
           return range_limit(risk_allocation_limit["risky"]["high"],allocation)
       elif mainstock in moderate :#imported from data
           return range_limit(risk_allocation_limit["risky"]["moderate"],allocation)
       elif mainstock in low :#imported from data
           return range_limit(risk_allocation_limit["risky"]["low"],allocation)
    elif portfolio_type == "moderate":
       if mainstock in Risky :#imported from data
           return range_limit(risk_allocation_limit["moderate"]["high"],allocation)
       elif mainstock in moderate :#imported from data
           return range_limit(risk_allocation_limit["moderate"]["moderate"],allocation)
       elif mainstock in low :#imported from data
           return range_limit(risk_allocation_limit["moderate"]["low"],allocation)
    elif portfolio_type == "lowrisk":
       if mainstock in Risky :#imported from data
           return range_limit(risk_allocation_limit["lowrisk"]["high"],allocation)
       elif mainstock in moderate :#imported from data
           return range_limit(risk_allocation_limit["lowrisk"]["moderate"],allocation)
       elif mainstock in low :#imported from data
           return range_limit(risk_allocation_limit["lowrisk"]["low"],allocation)






def diversification_limit_violated(allocation_stock, portfolio_type,allocation):
   diversification_limit = {
         "risky":   (0.00 , 0.25),
         "moderate":   (0.00 , 0.15),
         "lowrisk":   (0.00 , 0.10),
         }
   if portfolio_type=="risky":
       return range_limit(diversification_limit["risky"] ,allocation)
   elif portfolio_type=="moderate":
       return range_limit(diversification_limit["moderate"] ,allocation)
   elif portfolio_type=="lowrisk":
       return range_limit(diversification_limit["lowrisk"] ,allocation)





def range_limit (tuple, value):
  x,y = tuple
  if x <= value <= y:
     return True
  else:
     return False




def expand(stocks , mainstock, portfolio , user_constraints):
    expand = []
    default  = {"id": mainstock.symbol, "hold": 0}
    expand.append(default)
    buy_portfolio=None
    sell_portfolio=None
    with open('data/multiple_stocks_ratios.json', 'r') as file:
        data = json.load(file)
    for stockinfo in data :
       if stockinfo["symbol"] == mainstock.ticker:
           for stock in portfolio.stocks :
               if stock.stock == mainstock.symbol:
                  buy_portfolio=copy.deepcopy(portfolio)
                  stock.amt-=2
                  if stock.amt >= 0 :
                      sell_portfolio = copy.deepcopy(portfolio)
                  stock.amt+=1 #bacak to its orignal value (we can use the deep copy method)
    stat_variables = GetPortfolioStatVariables(buy_portfolio,)
    std_value = stat_variables["standard_deviation"]
    var = stat_variables["Var"]
    cvar = stat_variables["Cvar"]
    #-----------------------------------------
    #Buy action check
    #-----------------------------------------
    if portfolio.funds >= mainstock.value  :
       w= GetWeight(mainstock , portfolio);

    #diversification and stock class check
    portfolio_type = user_constraints["preffered_risk"]
    #Risk check
    portfolio_risk = CalculateTotalRisk(std_value,var,cvar)
    if portfolio_risk == user_constraints["preferred_risk"] and diversification_limit_violated(w,portfolio) and stock_class_limit_violated( Risky ,moderate ,low , portfolio_type, mainstock):
        expand.append({"id":mainstock.symbol,"buy":-mainstock.value})

    #-----------------------------------------
    #Sell action check
    #-----------------------------------------
    if sell_portfolio :
       portfolio_risk = CalculateTotalRisk(std_value,var,cvar)
       if portfolio_risk == user_constraints["preferred_risk"]  and diversification_limit_violated(w,portfolio) and stock_class_limit_violated( Risky ,moderate ,low , portfolio_type, mainstock):
           expand.append({"id":mainstock.symbol,"sell":+mainstock.value})

    return expand




def classify_stock_risk(portfolio):
    Risky=[]
    Moderate=[]
    LowRisk=[]

    with open ('data/multiple_stocks_ratios.json', 'r') as f :
       data = json.load(f)

    vol_list= extract_ratio_list (data,"volatility")
    cu_list= extract_ratio_list (data,"currentRatioTTM")
    dept_list= extract_ratio_list (data,"dept_to_equity")


    for stockitem in portfolio.stocks :
        stock_ratios = next((item for item in data if item["symbol"]==stockitem.stock),None)
        volatility = stock_ratios.get("volatility", 1.0)
        cu_ratio = stock_ratios.get("currentRatioTTM",15.0)
        debt_equity = stock_ratios.get("dept_to_equity",1.0)

        vol_nor = normalize(volatility,min(vol_list),max(vol_list))
        cu_nor = normalize(cu_ratio,min(cu_list),max(cu_list))
        dept_nor = normalize(debt_equity,min(dept_list),max(dept_list))

        class_val = vol_nor*0.5-cu_nor*0.2+0.3*dept_nor

        if class_val<0.3:
          LowRisk.append(stockitem)
        elif class_val<0.6:
          Moderate.append(stockitem)
        else:
          Risky.append(stockitem)
    return Risky,Moderate,LowRisk




def extract_ratio_list(data, key ,default=0.0 ):
    return [item.get(key, default) for item in data if item.get(key) is not None   ]



def normalize (value, min , max):
  if min == max:
     return 0.0
  return (value -min) / (max - min)

def get_top_short_term(df, top_n=10):
    df2 = df.dropna(subset=["expected_return", "volatility", "1M_pct_change"]).copy()
    df2.loc[:, "score"] = (
        df2["expected_return"].rank(ascending=False) +
        df2["1M_pct_change"].rank(ascending=False) +
        df2["volatility"].rank(ascending=True)
    ).round(6)
    df3 = df2.reset_index().sort_values(
        by=["score", "symbol"],
        ascending=[False, True],
        kind="mergesort"
    )
    return df3["symbol"].tolist()[:top_n]


def get_top_long_term(df, top_n=10):
    df2 = df.dropna(subset=[ #
        "returnOnEquityTTM",
        "freeCashFlowPerShareTTM",
        "netProfitMarginTTM",
        "pegRatioTTM",
        "debtRatioTTM"
    ]).copy()
    df2.loc[:, "score"] = (
        df2["returnOnEquityTTM"].rank(ascending=False) +
        df2["freeCashFlowPerShareTTM"].rank(ascending=False) +
        df2["netProfitMarginTTM"].rank(ascending=False) +
        df2["pegRatioTTM"].rank(ascending=True) +
        df2["debtRatioTTM"].rank(ascending=True)
    ).round(6)
    df3 = df2.reset_index().sort_values(
        by=["score", "symbol"],
        ascending=[False, True],
        kind="mergesort"
    )
    return df3["symbol"].tolist()[:top_n]





def test_expand():
    # Dummy Stock class
    class Stock:
        def __init__(self, symbol, price, shares):
            self.symbol = symbol
            self.ticker = symbol  # Needed by `expand()`
            self.stock = symbol   # Needed by loop in `expand()`
            self.price = price
            self.shares = shares
            self.amt = shares

    # Dummy Portfolio class
    class Portfolio:
        def __init__(self, stocks, funds):
            self.stocks = stocks
            self.funds = funds

    # Create sample stock and portfolio
    main_stock = Stock("AAPL", 150.0, 10)
    another_stock = Stock("GOOGL", 2800.0, 2)
    portfolio = Portfolio([main_stock, another_stock], 10000.0)

    # Fake user constraints
    user_constraints = {}

    # Create a mock file with expected format
    with open('data/multiple_stocks_ratios.json', 'w') as f:
        json.dump([{"symbol": "AAPL", "some_other_data": "value"}], f)

    # Call expand function
    result = expand([main_stock, another_stock], main_stock, portfolio, user_constraints)

    # Print result
    print("Expand Result:")
    for node in result:
        print(node)

# Run the test
test_expand()



FileNotFoundError: [Errno 2] No such file or directory: 'data/multiple_stocks_ratios.json'